In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import time
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
df=pd.read_excel(r"../dataset/data.xlsx",sheet_name="Sheet1")
df

,Number of Storeys(NS),Height of Structure(HS),Number of Spans(HSP),Length of Spans (LS),Opening percentage (OP),Masonry wall Stiffeness Et (MS),Period (P)
0,1,3,2,3.0,0,2.25,0.092970
1,1,3,2,3.0,100,2.25,0.159750
2,1,3,2,4.5,0,2.25,0.094770
3,1,3,2,4.5,100,2.25,0.156589
4,1,3,2,6.0,0,2.25,0.106350
...,...,...,...,...,...,...,...
4021,22,66,6,6.0,50,25.00,1.628000
4022,22,66,6,6.0,75,25.00,2.714000
4023,22,66,6,6.0,100,25.00,2.879000
4024,22,66,6,7.5,0,25.00,0.602680


In [3]:
df.shape

(4026, 7)

In [4]:
df.head()

,Number of Storeys(NS),Height of Structure(HS),Number of Spans(HSP),Length of Spans (LS),Opening percentage (OP),Masonry wall Stiffeness Et (MS),Period (P)
0,1,3,2,3.0,0,2.25,0.092970
1,1,3,2,3.0,100,2.25,0.159750
2,1,3,2,4.5,0,2.25,0.094770
3,1,3,2,4.5,100,2.25,0.156589
4,1,3,2,6.0,0,2.25,0.106350


In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df)
normalized_df = pd.DataFrame(normalized_data, columns=df.columns)

print(normalized_df)

      Number of Storeys(NS)  Height of Structure(HS)  Number of Spans(HSP)  \
0                       0.0                      0.0                   0.0   
1                       0.0                      0.0                   0.0   
2                       0.0                      0.0                   0.0   
3                       0.0                      0.0                   0.0   
4                       0.0                      0.0                   0.0   
...                     ...                      ...                   ...   
4021                    1.0                      1.0                   1.0   
4022                    1.0                      1.0                   1.0   
4023                    1.0                      1.0                   1.0   
4024                    1.0                      1.0                   1.0   
4025                    1.0                      1.0                   1.0   

      Length of Spans (LS)  Opening percentage (OP)  \
0       

In [6]:
normalized_df.keys()

Index(['Number of Storeys(NS)', 'Height of Structure(HS)',
       'Number of Spans(HSP)', 'Length of Spans (LS)',
       'Opening percentage (OP)', 'Masonry wall Stiffeness Et (MS)',
       'Period (P)'],
      dtype='object')

In [7]:
X=normalized_df.drop("Period (P)" ,axis=1)

In [8]:
y=normalized_df["Period (P)"]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=99)

# SVM

In [10]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,cross_validate
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
from hyperopt import hp,fmin,tpe,Trials,partial
from hyperopt.early_stop import no_progress_loss
import hyperopt

In [11]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import KFold,cross_validate


In [20]:
def bayesopt_objective(gamma,C):
    model=SVR(gamma=int(gamma)
              ,C=int(C))
    cv=KFold(n_splits=10,shuffle=True,random_state=99)
    validate_loss=cross_validate(model,X_train,y_train
                                   ,cv=cv
                                   ,scoring='neg_root_mean_squared_error'
                                   ,n_jobs=-1
                                   ,error_score='raise')
    
    return np.mean(abs(validate_loss['test_score']))

In [21]:
def param_bayes_opt(init_points,n_iter):
    opt=BayesianOptimization(bayesopt_objective
                             ,param_grid_simple
                             ,random_state=99)

    opt.maximize(init_points=init_points 
                 ,n_iter=n_iter 
                )
    

    params_best=opt.max['params'] 
    score_best=opt.max['target'] 
    
   
    print("\n","best params: ", params_best,
          "\n","best cvscore: ", score_best)
    
    return params_best,score_best

In [22]:
def bayes_opt_validation(params_best):
    model=SVR(gamma=int(params_best['gamma'])
            ,C=int(params_best['C'])
           )
    cv=KFold(n_splits=10,shuffle=True,random_state=99)
    validate_loss=cross_validate(model,X_train,y_train
                                   ,cv=cv
                                   ,scoring='neg_root_mean_squared_error'
                                   ,n_jobs=-1
                                  )
    
    return np.mean(abs(validate_loss['test_score']))

In [23]:
# Start time
start_time = time.time()

param_grid_simple={'gamma':(0.01,1)
                  ,'C':(1,10)
                  }

params_best,score_best=param_bayes_opt(20,100)

params_best 
score_best
validation_score=bayes_opt_validation(params_best)
validation_score 
# End time
end_time = time.time()
# Calculate total time
total_time = end_time - start_time
print("Total time taken: ", total_time)

|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
| 1         | 0.2244    | 7.051     | 0.4932    |
| 2         | 0.2244    | 8.429     | 0.04113   |
| 3         | 0.2244    | 8.272     | 0.57      |
| 4         | 0.2244    | 3.679     | 0.05623   |
| 5         | 0.2244    | 9.916     | 0.01676   |
| 6         | 0.2244    | 7.928     | 0.7493    |
| 7         | 0.2244    | 4.397     | 0.4992    |
| 8         | 0.2244    | 9.361     | 0.4015    |
| 9         | 0.2244    | 9.766     | 0.5292    |
| 10        | 0.2244    | 1.843     | 0.8152    |
| 11        | 0.2244    | 2.905     | 0.5588    |
| 12        | 0.2244    | 3.63      | 0.818     |
| 13        | 0.2244    | 8.452     | 0.2294    |
| 14        | 0.2244    | 6.804     | 0.1042    |
| 15        | 0.2244    | 4.705     | 0.1059    |
| 16        | 0.2244    | 2.296     | 0.2201    |
| 17        | 0.2244    | 5.29      | 0.08684   |
| 18        | 0.2244    | 3.115     | 0.01649   |


| 100       | 0.2244    | 5.137     | 0.8012    |
| 101       | 0.2244    | 7.151     | 0.2771    |
| 102       | 0.2244    | 7.242     | 0.1275    |
| 103       | 0.2244    | 5.209     | 0.596     |
| 104       | 0.2244    | 9.342     | 0.1513    |
| 105       | 0.2244    | 7.602     | 0.143     |
| 106       | 0.2244    | 7.186     | 0.7182    |
| 107       | 0.2244    | 3.564     | 0.8676    |
| 108       | 0.2244    | 2.921     | 0.1983    |
| 109       | 0.2244    | 8.898     | 0.9997    |
| 110       | 0.2244    | 4.127     | 0.8229    |
| 111       | 0.2244    | 8.758     | 0.793     |
| 112       | 0.2244    | 3.221     | 0.24      |
| 113       | 0.2244    | 6.231     | 0.4093    |
| 114       | 0.2244    | 3.333     | 0.9493    |
| 115       | 0.2244    | 6.498     | 0.0615    |
| 116       | 0.2244    | 5.981     | 0.8227    |
| 117       | 0.2244    | 7.201     | 0.5506    |
| 118       | 0.2244    | 4.734     | 0.4682    |
| 119       | 0.2244    | 4.906     | 0.9437    |


In [24]:
from sklearn.svm import SVR
params_best['C'] = int(params_best['C'])
params_best['gamma'] = int(params_best['gamma'])
svr_reg=SVR(**params_best )
svr_reg.fit(X_train,y_train)

SVR(C=7, gamma=0.4932)

In [25]:
y_pred=svr_reg.predict(X_test)
y_pred1=svr_reg.predict(X_val)
y_pred2=svr_reg.predict(X_train)

In [26]:
print("mean_absolute_error:", mean_absolute_error(y_val, y_pred1))
print("mean_squared_error:", mean_squared_error(y_val, y_pred1))
print("rmse:", sqrt(mean_squared_error(y_val, y_pred1)))
print("r2 score:", r2_score(y_val, y_pred1))

mean_absolute_error: 0.03893501645269835
mean_squared_error: 0.0022024947810892577
rmse: 0.046930744518804066
r2 score: 0.9558253639946267


In [27]:
print("mean_absolute_error:", mean_absolute_error(y_train, y_pred2))
print("mean_squared_error:", mean_squared_error(y_train, y_pred2))
print("rmse:", sqrt(mean_squared_error(y_train, y_pred2)))
print("r2 score:", r2_score(y_train, y_pred2))

mean_absolute_error: 0.03896861156760132
mean_squared_error: 0.0021703640457400274
rmse: 0.04658716610548475
r2 score: 0.9556301554885444


In [28]:
print("mean_absolute_error:", mean_absolute_error(y_test, y_pred))
print("mean_squared_error:", mean_squared_error(y_test, y_pred))
print("rmse:", sqrt(mean_squared_error(y_test, y_pred)))
print("r2 score:", r2_score(y_test, y_pred))

mean_absolute_error: 0.03739272261263716
mean_squared_error: 0.002022179196284038
rmse: 0.04496864681401963
r2 score: 0.9605279523278474
